In [ ]:
import pandas as pd
import numpy as np
import itertools
import pytest
import platform
import pickle
pd.set_option('display.max_rows',100)

# Part 1 - Create aggregated data
This section creates the aggregated and anonymised data directly from the raw data. This allows using make_table(raw=False) which creates summary tables from the raw data, before it has been anonymised.

#### Read in raw data

In [ ]:
if platform.system() == 'Darwin':
    shared_drive = '/Volumes/Data/EAU/Statistics/'
elif platform.system() == 'Windows':
    shared_drive = 'G:/'
# os.path.join(mydir, myfile)

raw_data_dir = shared_drive + 'Economic Estimates/Employment - Helen/max-csv-data/'
#raw_data_dir = '~/data/'

years = range(2011, 2017 + 1)

raw_data = {y:pd.read_csv(raw_data_dir +  'raw_' + str(y) + "_df.csv") for y in years}

#### Make lookups

In [ ]:
regionlookupdata = pd.read_csv('region-lookup.csv')
regionlookdict = {row[0]: row[1] for index, row in regionlookupdata.iterrows()}

sic_mappings = pd.read_csv("sic_mappings.csv")
sic_mappings = sic_mappings[sic_mappings.sic != 62.011]
sic_mappings.sic = round(sic_mappings.sic * 100, 0)
sic_mappings.sic = sic_mappings.sic.astype(int)

#### Clean and aggregate data

In [ ]:
demographics = ['sex', 'ethnicity', 'dcms_ageband', 'ftpt', 'nssec']
other_vars = ['sector', 'emptype', 'year', 'region']

def expand_grid(data_dict):
   rows = itertools.product(*data_dict.values())
   return pd.DataFrame.from_records(rows, columns=data_dict.keys())

def clean_data(year):
    
    # find weighting column name for given year    
    if year < 2016:
        weightedcountcol = 'PWTA14'
    if year == 2016:
        weightedcountcol = 'PWTA16'
    if year == 2017:
        weightedcountcol = 'PWTA17'

    df = raw_data[year]
    
    df['regionmain'] = df.GORWKR.map(regionlookdict)
    df['regionsecond'] = df.GORWK2R.map(regionlookdict)
    df['ftpt'] = df['ftpt'].astype(str)
    df['nssec'] = df['nssec'].astype(str)
    
    catuniques = []
    for caty in demographics + ['region']:
        if caty == 'region':
            catuniques.append(np.unique(regionlookupdata.mapno))
        else:
            catuniques.append(np.unique(df[caty]))
    
    x = pd.Series(np.unique(sic_mappings.sector))
    y = pd.Series(["civil_society", "total_uk", "overlap"])
    x = x.append(y)
    
    aggdict = {}
    aggdict['sector'] = x
    for caty in demographics + ['region']:
        if caty == 'region':
            aggdict[caty] = np.unique(regionlookupdata.mapno)
        else:
            aggdict[caty] = np.unique(df[caty])

    agg = expand_grid(aggdict)

    for subset in ['mainemp', 'secondemp', 'mainselfemp', 'secondselfemp']:
        if subset == 'mainemp':
            sicvar = "INDC07M"
            emptype = "INECAC05"
            emptypeflag = 1
            regioncol = 'regionmain'
    
        if subset == 'secondemp':
            sicvar = "INDC07S"
            emptype = "SECJMBR"
            emptypeflag = 1
            regioncol = 'regionsecond'
    
        if subset == 'mainselfemp':
            sicvar = "INDC07M"
            emptype = "INECAC05"
            emptypeflag = 2
            regioncol = 'regionmain'
    
        if subset == 'secondselfemp':
            sicvar = "INDC07S"
            emptype = "SECJMBR"
            emptypeflag = 2
            regioncol = 'regionsecond'

        # create subset for each of 4 groups
        df['region'] = df[regioncol]
        df['region'] = df['region'].fillna('missing region')
        dftemp = df[[sicvar, emptype, weightedcountcol, 'cs_flag'] + demographics + ['region']].copy()
        dftemp = dftemp.loc[dftemp[emptype] == emptypeflag]
        # need separate sic column to allow merging - I think
        dftemp.rename(columns={sicvar : 'sic'}, inplace=True)

        # total uk includes missing sics, so take copy before removing missing sics
        dftemp_totaluk = dftemp.copy()
        
        # remove rows from subset with missing sic
        dftemp = dftemp.loc[np.isnan(dftemp.sic) == False]
        
        # add sector column and further subset to all sectors excluding all_dcms
        dftemp_sectors = pd.merge(dftemp, sic_mappings.loc[:,['sic', 'sector']], how = 'inner')
        dftemp_sectors = dftemp_sectors.loc[dftemp_sectors['sector'] != 'all_dcms']
        
        # subset civil society
        dftemp_cs = dftemp.loc[dftemp['cs_flag'] == 1].copy()
        dftemp_cs['sector'] = 'civil_society'
        dftemp_cs = dftemp_cs[dftemp_sectors.columns.values]
        
        # subset all_dcms (still need to add cs and remove overlap)
        dftemp_all_dcms = pd.merge(dftemp, sic_mappings.loc[:,['sic', 'sector']], how = 'inner')
        dftemp_all_dcms = dftemp_all_dcms.loc[dftemp_all_dcms['sector'] == 'all_dcms']
        
        # subset overlap between sectors
        dftemp_all_dcms_overlap = pd.merge(dftemp, sic_mappings.loc[:,['sic', 'sector']], how = 'inner')
        dftemp_all_dcms_overlap = dftemp_all_dcms_overlap.loc[dftemp_all_dcms_overlap['sector'] == 'all_dcms']
        dftemp_all_dcms_overlap = dftemp_all_dcms_overlap.loc[dftemp_all_dcms_overlap['cs_flag'] == 1]
        dftemp_all_dcms_overlap['sector'] = 'overlap'
        
        # subset uk total
        dftemp_totaluk['sector'] = 'total_uk'
        # reorder columns
        dftemp_totaluk = dftemp_totaluk[dftemp_sectors.columns.values]
        
        # append different subsets together
        dftemp = dftemp_totaluk.append(dftemp_sectors).append(dftemp_cs).append(dftemp_all_dcms).append(dftemp_all_dcms_overlap)
        
        # this converts sic back to numeric
        dftemp = dftemp.infer_objects()
        
        # only total_uk sector has nan sics so groupby is dropping data - setting missing values to 'missing'
        dftemp['sic'] = dftemp['sic'].fillna(value=-1)

        # create column with unique name (which is why pd.DataFrame() syntax is used) which sums the count by sector
        aggtemp = pd.DataFrame({subset : dftemp.groupby( ['sector', 'sic'] + demographics + ['region'])[weightedcountcol].sum()}).reset_index()
        
        # merge final stacked subset into empty dataset containing each sector and category level combo
        # should be able to just use aggtemp for first agg where subset=='mainemp', but gave error, need to have play around. checking that agg has all the correct sectors and cat levels should be a separate piece of code.
        agg = pd.merge(agg, aggtemp, how='outer')
     
    agg['year'] = year
    return agg

def clean_data2(df):
    
    # fill in missing values to avoid problems with NaN
    agg = df.fillna(0)
    
    # sum main and second jobs counts together
    agg['emp'] = agg['mainemp'] + agg['secondemp']
    agg['selfemp'] = agg['mainselfemp'] + agg['secondselfemp']
    agg.drop(['mainemp', 'secondemp', 'mainselfemp', 'secondselfemp'], axis=1, inplace=True)
    
    # melt 'emp' and 'selfemp' into 'emptype'
    agg = agg[['sector', 'sic', 'year', 'emp', 'selfemp', 'region'] + demographics]
    melted = pd.melt(agg, id_vars=['sector', 'sic', 'year', 'region'] + demographics, var_name='emptype', value_name='count')  
    
    # need to aggregate before we can add civil society to all_dcms?
    
    # reduce down to desired aggregate
    aggfinal = melted.drop(['sic'], axis=1)
    aggfinal = aggfinal.groupby(['sector', 'emptype', 'year', 'region'] + demographics).sum()
    aggfinal = aggfinal.reset_index(['sector', 'emptype', 'year', 'region'] + demographics)
    
    # add civil society to all_dcms and remove overlap from all_dcms
    aggfinaloverlap = aggfinal.copy()
    
    alldcmsindex = aggfinaloverlap[aggfinaloverlap['sector'] == 'all_dcms'].index
    csindex = aggfinaloverlap[aggfinaloverlap['sector'] == 'civil_society'].index
    overlapindex = aggfinaloverlap[aggfinaloverlap['sector'] == 'overlap'].index
    newalldcms = aggfinaloverlap.loc[alldcmsindex, ['count']].reset_index(drop=True) + aggfinaloverlap.loc[csindex, ['count']].reset_index(drop=True) - aggfinaloverlap.loc[overlapindex, ['count']].reset_index(drop=True)
    newalldcms2 = newalldcms['count']
    newalldcms3 = np.array(newalldcms2)
    aggfinaloverlap.loc[alldcmsindex, ['count']] = newalldcms3
    
    return aggfinaloverlap

#### Run `clean_data()` on each year's data and append

In [ ]:
cleaned_data = [clean_data(i) for i in years]
agg = pd.concat(cleaned_data, ignore_index=True)

In [ ]:
list(agg.columns.values)

#### Run `clean_data2()` to convert the 'emp' columns into a single 'emptype' and 'count'

In [ ]:
aggfinal = clean_data2(agg)

In [ ]:
list(aggfinal.columns.values)

# Part 2 - Update individual aggregated and anonymised CSVs

This creates aggregated, anonymised csv for each possible table and saves them in the directory agg_csvs/

In [ ]:
from itertools import chain, combinations
# find all powersets - from https://docs.python.org/3/library/itertools.html#itertools-recipes
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# create list of all combinations of 
make_table_combos = [list(x) for x in powerset(demographics + other_vars)]

# define function to aggregate and anonymise data, then save to csv
def reduce_and_save_data(combo):
    mydf = aggfinal.groupby(combo)['count'].sum().reset_index()
    mydf.loc[mydf['count'] < 6000, 'count'] = 0    
    mydf.to_csv('agg_csvs/' + '_'.join(sorted(combo)) + '.csv')

# run function for each combination
for com in make_table_combos[1:]:
    reduce_and_save_data(com)


# Part 3 - Making Tables
This package contains aggregated, anonymised data which can be used to create summary tables. To generate/reresh this data, or produce un-anonymised summary tables see part 2 for which you will require access to the underlying raw data.

#### Define function to create summary table

In [ ]:
def make_table(index, columns, sub_col=None, sub_value=None, raw=False):
    
    # create flag for whether data needs to be subsetted to all_dcms
    if 'sector' not in index and 'sector' not in columns and 'sector' not in [sub_col]:
        all_dcms = True
    else:
        all_dcms = False
        
    if raw:
        agg_temp = aggfinal
        
    else:
        # user specified subset data
        if sub_col:
            all_cols = index + columns + [sub_col]
        else:
            all_cols = index + columns
        if all_dcms:
            all_cols = all_cols + ['sector']
            
        agg_temp = pd.read_csv('agg_csvs/' + '_'.join(sorted(all_cols)) + '.csv')
    
    # subset data if subset arguments provided
    if sub_col:
            agg_temp = agg_temp.loc[agg_temp[sub_col] == sub_value]
        
    # for non sector breakdowns, subset data to only inlcude 'all_dcms' sector
    if all_dcms:
        agg_temp = agg_temp.loc[agg_temp.sector == 'all_dcms']
        
    # pd.crosstab() only accepts lists of series not subsetted dataframes
    sindex = [agg_temp[col] for col in index]
    scolumns = [agg_temp[col] for col in columns]    
    # create table
    tb = pd.crosstab(index=sindex, columns=scolumns, values=agg_temp['count'], aggfunc='sum')
    
    # reorder columns and index
    orderings = {
        'sector': ["civil_society", "creative", "culture", "digital", "gambling", "sport", "telecoms", "all_dcms", "total_uk"],
        'sex': ['Male', 'Female'],
        'region': ['North East', 'North West', 'Yorkshire and the Humber', 'East Midlands', 'West Midlands', 'East of England', 'London', 'South East', 'South West', 'Wales', 'Scotland', 'Northern Ireland'],
    }
    for i in [i for i in index if i in orderings]:
        if isinstance(tb.index, pd.core.index.MultiIndex):
            tb = tb.reindex(orderings[i], axis=0, level=i)
        else:
            tb = tb.reindex(orderings[i], axis=0)

    for i in [i for i in columns if i in orderings]:
        if isinstance(tb.columns, pd.core.index.MultiIndex):
            tb = tb.reindex(orderings[i], axis=1, level=i)
        else:
            tb = tb.reindex(orderings[i], axis=1)
    
    
    # round and convert to 000's
    tb = round(tb / 1000, 0).astype(int)
    
    return tb


To make summary tables without having access to the raw data, we need aggregated tables with columns for each aspect of the breakdown in the table. For example, if we wanted to run `make_table(['sector'], ['sex'], 'year', 2016)` then we would need an aggregate table with columns for sector, sex, and year. This package stores each possible table as a csv and reads them in as required. For example, for `make_table(['sector'], ['sex'], 'year', 2016)` we have:

In [ ]:
pd.read_csv('agg_csvs/sector_sex_year.csv')

This is the data used to create `make_table(['sector'], ['sex'], 'year', 2016)`:

To refresh/update this data, run Part 2 followed by Part 3.

#### Other table examples

#### Region by employment type for 2017 data

In [ ]:
make_table(['emptype', 'sector'], ['region'], 'year', 2017)

#### Sector, employment type, and ethnicity time series

In [ ]:
eth_table = make_table(['ethnicity', 'sector'], ['year'])
eth_table.to_csv('deleteme.csv')

#### Make summary tables without anonymisation

In [ ]:
make_table(['ethnicity'], ['region', 'emptype'], 'year', 2017)

In [ ]:
make_table(['ethnicity'], ['region', 'emptype'], 'year', 2017, raw=True)

In [ ]:
make_table(['emptype', 'region'], 'year', 2017)

In [ ]:
make_table(['emptype', 'sector'], ['region'], 'year', 2017, raw=True)